# Pipeline: 
### EndNote Lib. >> .xml >> postgreSQL >> (Búsqueda DOI)+INSERT >> .enw >> EndNote Lib.

Control de etapas del *pipeline* para completado de DOI de la biblioteca de EndNote.

**Notas:**  
...

## Módulos

In [1]:
import sys
import os
import io 

from extraer_desde_xml.extrac_xml_to_df import extr_opc2
import carga_posgres.load as db
from completar_doi.add_doi import buscar_doi_v0
import temporizador as temp


# Cambiar la codificación de la salida estándar a UTF-8
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')


## Extracción desde .xml
El archivo .xml es generado desde EndNote con todos los registros de la biblioteca.  
La lógica para la extracción de las referencias almacenadas a registros de un *dataframe* se importa desde [extrac_xml_to_df.py](extraer_desde_xml\extrac_xml_to_df.py).

In [ ]:

df = extr_opc2('extraer_desde_xml\Endnote 20-10-22.xml')
print(df.head(10))

## Carga a base de datos (postgreSQL)
Para manipular los datos, estos se cargan en una base de datos relacional. Las operaciones previas necesarias se guardaron en el [Historial completo de Consultas SQL](sql_history.sql).

La lógica para la conexión con base de datos y la carga de los registros se importa desde [load.py](carga_posgres\load.py).

In [ ]:
# Asegurar tabla de destino
db.tabla_referencias()

# Carga de dataframe a postgres
db.load_all(df, True)

## Completar DOIs faltantes
Las funciones de búsqueda (empleando [API de *Crossref*](https://search.crossref.org/)) se importan desde [add_doi.py](completar_doi\add_doi.py)

Debido a las complicaciones particulares de cada tipo de referencia, este paso se realiza por separado para los **tipo = "Journal Article"**.

### DOIs para registros de "Journal Article"

In [ ]:
# DOI solo relevante luego de los 2000
resp = db.query_sql('''
    select nregistro, titulo from endnote.referencias 
        where 
            año > 2000 and
            doi is null and
            tipo = 'Journal Article';
''', cerrar = False)

df_sindoi = db.registros_a_df(resp, ["nregistro", "titulo"])

titulos = list(df_sindoi['titulo'])

def map_doi(tit):
    res = buscar_doi_v0(
        titulo = tit, 
        nitems = 10,
        terminal= True
    )
    return res['DOI'] if res else 'no hallado'

dois = list(map(lambda t: map_doi(t), titulos))
df_sindoi.insert(2, "doi_nuevo", dois)

db.query_sql('''
    create table if not exists endnote.busqueda_doi (
        nregistro INTEGER NOT NULL PRIMARY KEY,        
        titulo VARCHAR(440),
        doi_nuevo VARCHAR(125)
        );''', 
        cerrar = False
)

# INSERTAR en tabla "busqueda_doi"

db.load_to(df_sindoi, db.ESQUEMA, "busqueda_doi")

# Pasar a tabla "referencias" los DOI descargados
db.query_sql('''
        UPDATE endnote.referencias r
        SET doi = bd.doi_nuevo
        from   (select nregistro, doi_nuevo
                from endnote.busqueda_doi
                where doi_nuevo != 'no hallado') as bd
        where 
                bd.nregistro = r.nregistro;''', 
        cerrar = False
)


## Creación de 